In [1]:
import sys
sys.path.append('..')
from laspy.file import File
import pclpy
import pcl
import VelLoader
import ARUSTUFF
import Plane
import Corners
import cv2
import imp
import glob
import os
import numpy as np
import open3d
import pdal
import pandas as pd
import pclpy
import segTree
from functools import reduce

WARNING (<module>): Couldn't load netifaces, some utils won't work


In [2]:
imp.reload(segTree)
imp.reload(VelLoader)

<module 'VelLoader' from '..\\VelLoader.py'>

In [3]:
reader = VelLoader.VVdataloader(r'D:/Documentos/ParaAutonomia/Python/Proyecto/NIST_12-12-19/2019-12-11-10-14-26_Velodyne-VLP-16-Data (Frame 1600 to 26600) (Frame 0 to 5000).pcap')

In [4]:
reader.getdataFrame(1)

NameError: name 'tic' is not defined

In [5]:
dataframe1 = pd.concat([reader.getdataFrame(1),reader.getdataFrame(2)]) 
#dataframe2 = pd.concat([reader.getdataFrame(3),reader.getdataFrame(4)])
#dataframe3 = pd.concat([reader.getdataFrame(9),reader.getdataFrame(10)]) 

In [6]:
imp.reload(segTree)

<module 'segTree' from '..\\segTree.py'>

In [7]:
stemsR1,models = segTree.findstemsLiDAR(dataframe1[['X','Y','Z']].values)
#stemsR2,models = segTree.findstemsLiDAR(dataframe2[['X','Y','Z']].values)
#stemsR3,models = segTree.findstemsLiDAR(dataframe3[['X','Y','Z']].values)
allstems1 = reduce(lambda x,y:x+y.tolist(),stemsR1,[])
#allstems2 = reduce(lambda x,y:x+y.tolist(),stemsR2,[])
#allstems3 = reduce(lambda x,y:x+y.tolist(),stemsR3,[])

In [8]:
allstems1 = reduce(lambda x,y:x+y.tolist(),stemsR1,[])
allstems1 = [p for i in stemsR1 for p in i]

In [9]:
Corners.open3dpaint([allstems1,dataframe1[['X','Y','Z']].values])

In [10]:
cloud1 = open3d.PointCloud()
cloud1.points = open3d.Vector3dVector(allstems1)
cloud2 = open3d.PointCloud()
cloud2.points = open3d.Vector3dVector(allstems2)
trans_init = np.array([[1,0,0,0],
                      [0,1,0,0],
                      [0,0,1,0],
                      [0,0,0,1]])
threshold = 0.2

reg_p2p = open3d.registration.registration_icp(
        cloud1, cloud2, threshold, trans_init,
        open3d.registration.TransformationEstimationPointToPoint())

NameError: name 'allstems2' is not defined

In [ ]:
reg_p2p.transformation
reg1 = cloud1.transform(reg_p2p.transformation)
Corners.open3dpaint([reg1.points,allstems2])

In [ ]:
frame = 10
transforms = []
trans_init = np.array([[1,0,0,0],
                      [0,1,0,0],
                      [0,0,1,0],
                      [0,0,0,1]])
transforms.append(trans_init)
dataframesAC = []

for i in range(10):
    frame += 2
    dataframe1 = pd.concat([reader.getdataFrame(frame),reader.getdataFrame(frame+1)]) 
    dataframe2 = pd.concat([reader.getdataFrame(frame+2),reader.getdataFrame(frame+3)]) 
    dataframesAC.append([dataframe1,dataframe2])
    
    stemsR1,models = segTree.findstemsLiDAR(dataframe1[['X','Y','Z']].values)
    Nogroundpoints,ground = segTree.FloorRemove(dataframe1[['X','Y','Z']].values)
    vground = segTree.voxelize(ground,1)
    allstems1 = reduce(lambda x,y:x+y.tolist(),stemsR1+[vground],[])

    stemsR2,models = segTree.findstemsLiDAR(dataframe2[['X','Y','Z']].values)
    Nogroundpoints2,ground2 = segTree.FloorRemove(dataframe2[['X','Y','Z']].values)
    vground2 = segTree.voxelize(ground2,1)
    allstems2 = reduce(lambda x,y:x+y.tolist(),stemsR2+[vground2],[])

    cloud1 = open3d.PointCloud()
    cloud1.points = open3d.Vector3dVector(allstems1)
    cloud2 = open3d.PointCloud()
    cloud2.points = open3d.Vector3dVector(allstems2)

    threshold = 0.2

    reg_p2p = open3d.registration.registration_icp(
            cloud1, cloud2, threshold, trans_init,
            open3d.registration.TransformationEstimationPointToPoint())
    transforms.append(reg_p2p.transformation)

In [76]:
reg1 = cloud1.transform(reg_p2p.transformation)
Corners.open3dpaint([reg1.points,allstems2])

In [14]:
dataframe = pd.concat([reader.getdataFrame(1),reader.getdataFrame(2)]) 

In [15]:
pointsXYZ = dataframe[['X','Y','Z']].values
Nogroundpoints,ground = segTree.FloorRemove(pointsXYZ)
flatpoints = np.hstack([Nogroundpoints[:,0:2],np.zeros_like(Nogroundpoints)[:,0:1]])

RRFpoints = segTree.RadiusOutlierRemoval(flatpoints)
notgoodpoints = Nogroundpoints[np.isnan(RRFpoints[:,0])]
goodpoints = Nogroundpoints[np.bitwise_not(np.isnan(RRFpoints[:,0]))]

cluster_list = segTree.EucladeanClusterExtract(goodpoints)
RGclusters = []
for i in cluster_list:
    ppclusters = segTree.RegionGrowing(i)
    RGclusters.append(ppclusters)

models = []
stemsR = []
for i in RGclusters:
    for p in i:
        indices, model = segTree.segment_normals(p)
        prop = len(p[indices])/len(p)
        if len(indices)>1 and prop>0. and np.arccos(np.dot([0,0,1],model[3:6]))<.6:
            points = p[indices]
            PC,_,_ = Plane.getPrincipalComponents(points)
            if PC[0]/PC[1]>10:
                stemsR.append(points)
                models.append(model)

In [18]:
allS = []
stems = []
segcloud = pcl.PointCloud()
models = []
for p in cluster_list:
    
    segcloud.from_array(p)
    cylseg = segcloud.make_segmenter_normals(ksearch=20)
    #cylseg = segcloud.make_segmenter()
    cylseg.set_optimize_coefficients(True)
    cylseg.set_model_type(pcl.SACMODEL_CYLINDER)
    #cylseg.set_model_type(pcl.SACMODEL_LINE)
    cylseg.set_method_type(pcl.SAC_RANSAC)
    cylseg.set_normal_distance_weight(0.0001)
    cylseg.set_max_iterations(1000)
    #cylseg.set_MaxIterations(1000000)
    cylseg.set_distance_threshold(.1)
    cylseg.set_radius_limits(0,0.5)

    indices, model = cylseg.segment()

    prop = len(segcloud.extract(indices).to_array())/len(segcloud.to_array())
    if len(indices)>1 and prop>0. and np.arccos(np.dot([0,0,1],model[3:6]))<1.: 
        stems.append(segcloud.extract(indices).to_array())
        models.append(model)
        allS.append(prop)

In [19]:
allS = []
stemsLine = []
segcloud = pcl.PointCloud()
models = []
for p in cluster_list:
    segcloud.from_array(p)
    cylseg = segcloud.make_segmenter_normals(ksearch=20)
    cylseg.set_optimize_coefficients(True)
    cylseg.set_model_type(pcl.SACMODEL_LINE)
    cylseg.set_method_type(pcl.SAC_RANSAC)
    cylseg.set_normal_distance_weight(0.0001)
    cylseg.set_max_iterations(10000)
    cylseg.set_distance_threshold(0.5)
    cylseg.set_radius_limits(0,0.5)

    indices, model = cylseg.segment()

    prop = len(segcloud.extract(indices).to_array())/len(segcloud.to_array())
    if len(indices)>1 and prop>0. and np.arccos(np.dot([0,0,1],model[3:6]))<1.: 
        stemsLine.append(segcloud.extract(indices).to_array())

In [20]:
allS = []
stemsLine1 = []
segcloud = pcl.PointCloud()
models = []
for p in cluster_list:
    segcloud.from_array(p)
    cylseg = segcloud.make_segmenter()
    cylseg.set_optimize_coefficients(True)
    #cylseg.set_model_type(pcl.SACMODEL_LINE)
    cylseg.set_model_type(pcl.SACMODEL_LINE)
    cylseg.set_method_type(pcl.SAC_RANSAC)
    cylseg.set_distance_threshold(1)
    cylseg.set_MaxIterations(10000)
    indices, model = cylseg.segment()

    prop = len(segcloud.extract(indices).to_array())/len(segcloud.to_array())
    if len(indices)>1 and prop>0. and np.arccos(np.dot([0,0,1],model[3:6]))<1.: 
        stemsLine1.append(segcloud.extract(indices).to_array())
        #models.append(model)
        #allS.append(prop)

In [22]:
import functools

In [23]:
allstemsLine1 = functools.reduce(lambda x,y : x + y.tolist(),stemsLine1,[])
allstemsLine = functools.reduce(lambda x,y : x + y.tolist(),stemsLine,[])
allstems = functools.reduce(lambda x,y : x + y.tolist(),stems,[])
allstemsR = functools.reduce(lambda x,y : x + y.tolist(),stemsR,[])
Corners.open3dpaint([allstemsR,np.array(allstemsLine)*1.02,np.array(allstemsLine1)*1.04])

In [33]:
Corners.PCL3dpaint([allstemsR,ground,np.array(allstemsLine1)*1.02])

In [29]:
allstemsLine

[[15.020025253295898, 26.859609603881836, -5.989962100982666],
 [15.148399353027344, 26.655818939208984, -7.088009834289551],
 [15.36046028137207, 27.01021957397461, -6.047967910766602],
 [15.245530128479004, 26.789533615112305, -4.888606548309326],
 [16.030298233032227, 28.110010147094727, -1.698094367980957],
 [15.080810546875, 26.586627960205078, -7.0664143562316895],
 [15.282404899597168, 26.905542373657227, -6.0227813720703125],
 [15.272390365600586, 26.85152244567871, -4.89924430847168],
 [16.20162582397461, 28.3316593170166, -0.5766814351081848],
 [15.510217666625977, 26.83837127685547, -6.033466339111328],
 [16.904064178466797, 29.240421295166016, 2.9586284160614014],
 [17.404321670532227, 30.045215606689453, 6.7573957443237305],
 [16.465789794921875, 28.39645004272461, -0.5799624919891357],
 [15.438458442687988, 26.74378776550293, -6.0105695724487305],
 [16.836496353149414, 29.1460018157959, 2.9485182762145996],
 [15.404114723205566, 26.64847755432129, -4.881723880767822],
 [1